# Task 1 — Data Extraction, Cleaning, EDA, Stationarity, and Risk Metrics

Assets:
- TSLA (high risk / high growth)
- BND (low risk bonds ETF)
- SPY (broad market, moderate risk)

Period: 2015-01-01 to 2026-01-15  
Risk-free rate assumption for Sharpe Ratio: **2% annual**.

In [ ]:
from __future__ import annotations

# Ensure the project root is on sys.path so `import src...` works when running from notebooks/
import sys
from pathlib import Path

project_root = Path.cwd().resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.config import (
    TICKERS, START_DATE, END_DATE,
    PRICE_COL, PRICES_PATH, RETURNS_PATH,
    TASK1_ADF_PATH, TASK1_RISK_PATH, TASK1_OUTLIERS_PATH,
    DEFAULT_VAR_LEVEL, RISK_FREE_RATE_ANNUAL, ANNUALIZATION_FACTOR
)
from src.data_fetch import fetch_yfinance_prices
from src.data_prep import clean_prices, add_daily_returns
from src.stationarity import adf_test
from src.risk_metrics import historical_var, sharpe_ratio, annualized_vol
from src.eda import rolling_volatility, detect_outliers_zscore, summary_stats

from src.io import save_parquet, load_parquet, save_csv

sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 5)

## 1) Extract historical financial data (YFinance)
We fetch TSLA, BND, SPY OHLCV data and store it in a tidy format.

In [ ]:
prices_raw = fetch_yfinance_prices(TICKERS, START_DATE, END_DATE)
prices_raw.head(), prices_raw.shape

In [ ]:
(prices_raw.groupby("asset")["date"]
 .agg(["min", "max", "count"])
 .sort_index())

## 2) Data cleaning & understanding
We ensure correct dtypes, handle missing values, remove duplicates, and create a cleaned dataset.

In [ ]:
prices = clean_prices(prices_raw)
prices.isna().sum()

In [ ]:
(prices.groupby("asset")["date"]
 .agg(["min", "max", "count"])
 .sort_index())

In [ ]:
save_parquet(prices, PRICES_PATH)
PRICES_PATH

### Basic statistics (per asset)

In [ ]:
stats = summary_stats(prices)
stats.head(20)

## 3) Compute daily returns (Adj Close)
We compute daily percentage returns from adjusted close.

In [ ]:
prices_ret = add_daily_returns(prices, price_col=PRICE_COL)
prices_ret.head()

In [ ]:
save_parquet(prices_ret, RETURNS_PATH)
RETURNS_PATH

# EDA Visualization 1 — Adjusted Close over time

In [ ]:
for asset, g in prices_ret.groupby("asset"):
    plt.plot(g["date"], g[PRICE_COL], label=asset)
plt.title("Adjusted Close Price (2015–2026)")
plt.xlabel("Date")
plt.ylabel("Adj Close")
plt.legend()
plt.show()

## EDA Visualization 2 — Daily returns over time

In [ ]:
for asset, g in prices_ret.groupby("asset"):
    plt.plot(g["date"], g["return"], label=asset, alpha=0.8)
plt.title("Daily Returns (pct change of Adj Close)")
plt.xlabel("Date")
plt.ylabel("Daily Return")
plt.legend()
plt.show()

## EDA Visualization 3 — Rolling volatility (20-day)

In [ ]:
window = 20
for asset, g in prices_ret.groupby("asset"):
    rv = rolling_volatility(g["return"], window=window)
    plt.plot(g["date"], rv, label=asset)
plt.title(f"{window}-Day Rolling Volatility of Daily Returns")
plt.xlabel("Date")
plt.ylabel("Rolling Std Dev")
plt.legend()
plt.show()

## 4) Outlier detection (extreme daily returns)
We flag days where |z-score| ≥ 3 within each asset.

In [ ]:
outliers = detect_outliers_zscore(prices_ret, return_col="return", z=3.0)
outliers.sort_values(["asset", "zscore"]).head(10), outliers.shape

In [ ]:
save_csv(outliers, TASK1_OUTLIERS_PATH)
TASK1_OUTLIERS_PATH

## 5) Stationarity testing (ADF)
ADF test is applied to:
- Price series (Adj Close): often non-stationary
- Return series: often closer to stationary

In [ ]:
rows = []
for asset, g in prices_ret.groupby("asset"):
    g = g.sort_values("date")
    res_price = adf_test(g[PRICE_COL])
    res_ret = adf_test(g["return"].dropna())

    rows.append({"asset": asset, "series": PRICE_COL, **res_price})
    rows.append({"asset": asset, "series": "return", **res_ret})

adf_results = pd.DataFrame(rows)
adf_results

In [ ]:
adf_export = adf_results.copy()
adf_export["crit_1pct"] = adf_export["critical_values"].apply(
    lambda d: d["1%"])
adf_export["crit_5pct"] = adf_export["critical_values"].apply(
    lambda d: d["5%"])
adf_export["crit_10pct"] = adf_export["critical_values"].apply(
    lambda d: d["10%"])
adf_export = adf_export.drop(columns=["critical_values"])

save_csv(adf_export, TASK1_ADF_PATH)
TASK1_ADF_PATH

### Interpretation (to complete)
- If p-value > 0.05 for prices, we fail to reject the unit-root hypothesis → non-stationary prices.
- If p-value < 0.05 for returns, we reject the unit-root hypothesis → (more) stationary returns.
- Implication for ARIMA: price series usually requires differencing (d ≥ 1).


## 6) Risk metrics (VaR, Sharpe)
We compute:
- Historical VaR at 95% confidence (loss threshold)
- Annualized volatility
- Sharpe ratio using risk-free rate = 2% annual


In [ ]:
risk_rows = []
for asset, g in prices_ret.groupby("asset"):
    r = g["return"]
    risk_rows.append({
        "asset": asset,
        "VaR_95": historical_var(r, level=DEFAULT_VAR_LEVEL),
        "ann_vol": annualized_vol(r, annualization_factor=ANNUALIZATION_FACTOR),
        "sharpe_rf_annual": RISK_FREE_RATE_ANNUAL,
        "sharpe": sharpe_ratio(r, rf_annual=RISK_FREE_RATE_ANNUAL, annualization_factor=ANNUALIZATION_FACTOR),
    })

risk_table = pd.DataFrame(risk_rows).sort_values("asset")
risk_table

In [ ]:
save_csv(risk_table, TASK1_RISK_PATH)
TASK1_RISK_PATH

## Data Quality Summary (to complete)
- Missing values: 0 (0%) across all cleaned fields ( date, asset, open, high, low, close, adj_close, volume).
- Handling method: Missing prices were forward-filled within each asset; missing volume set to 0; any remaining missing rows dropped. Final cleaned outputs contain no missing values.
- Duplicates: Duplicates defined on (asset, date)were removed (kept last), ensuring one record per asset per trading day.
- Coverage/consistency: All assets have aligned coverage from 2015-01-02 to 2026-01-14 with 2775 observations each.
- Notes/limitations: yfinance-sourced OHLCV/adjusted prices may contain provider revisions/anomalies; adjusted prices reflect corporate actions where applicable. Data is on trading days only; holidays and differing trading calendars can affect alignment when expanding the universe.
